# Train your first 🐸 TTS model 💫

### 👋 Hello and welcome to Coqui (🐸) TTS

The goal of this notebook is to show you a **typical workflow** for **training** and **testing** a TTS model with 🐸.

Let's train a very small model on a very small amount of data so we can iterate quickly.

In this notebook, we will:

1. Download data and format it for 🐸 TTS.
2. Configure the training and testing runs.
3. Train a new model.
4. Test the model and display its performance.

So, let's jump right in!


In [11]:
## Install Coqui TTS
! pip install -U pip
! pip install TTS

## ✅ Data Preparation

### **First things first**: we need some data.

We're training a Text-to-Speech model, so we need some _text_ and we need some _speech_. Specificially, we want _transcribed speech_. The speech must be divided into audio clips and each clip needs transcription. More details about data requirements such as recording characteristics, background noise and vocabulary coverage can be found in the [🐸TTS documentation](https://tts.readthedocs.io/en/latest/formatting_your_dataset.html).

If you have a single audio file and you need to **split** it into clips. It is also important to use a lossless audio file format to prevent compression artifacts. We recommend using **wav** file format.

The data format we will be adopting for this tutorial is taken from the widely-used  **LJSpeech** dataset, where **waves** are collected under a folder:

<span style="color:purple;font-size:15px">
/wavs<br />
 &emsp;| - audio1.wav<br />
 &emsp;| - audio2.wav<br />
 &emsp;| - audio3.wav<br />
  ...<br />
</span>

and a **metadata.csv** file will have the audio file name in parallel to the transcript, delimited by `|`:

<span style="color:purple;font-size:15px">
# metadata.csv <br />
audio1|This is my sentence. <br />
audio2|This is maybe my sentence. <br />
audio3|This is certainly my sentence. <br />
audio4|Let this be your sentence. <br />
...
</span>

In the end, we should have the following **folder structure**:

<span style="color:purple;font-size:15px">
/MyTTSDataset <br />
&emsp;| <br />
&emsp;| -> metadata.csv<br />
&emsp;| -> /wavs<br />
&emsp;&emsp;| -> audio1.wav<br />
&emsp;&emsp;| -> audio2.wav<br />
&emsp;&emsp;| ...<br />
</span>

🐸TTS already provides tooling for the _LJSpeech_. if you use the same format, you can start training your models right away. <br />

After you collect and format your dataset, you need to check two things. Whether you need a **_formatter_** and a **_text_cleaner_**. <br /> The **_formatter_** loads the text file (created above) as a list and the **_text_cleaner_** performs a sequence of text normalization operations that converts the raw text into the spoken representation (e.g. converting numbers to text, acronyms, and symbols to the spoken format).

If you use a different dataset format then the LJSpeech or the other public datasets that 🐸TTS supports, then you need to write your own **_formatter_** and  **_text_cleaner_**.

## ⏳️ Loading your dataset
Load one of the dataset supported by 🐸TTS.

We will start by defining dataset config and setting LJSpeech as our target dataset and define its path.


In [12]:
import os
from google.colab import files
import zipfile

print("=== Uploading MyTTSDataset ===")
print("Please select your MyTTSDataset.zip file from:")
print("C:\\Users\\shafe\\OneDrive\\shafeeq\\Saras AI\\Project- TTS- voice cloning\\MyTTSDataset.zip")
print("\n")

=== Uploading MyTTSDataset ===
Please select your MyTTSDataset.zip file from:
C:\Users\shafe\OneDrive\shafeeq\Saras AI\Project- TTS- voice cloning\MyTTSDataset.zip




In [25]:
# Download and extract LJSpeech dataset.
output_path = "/content"
!wget -O $output_path/LJSpeech-1.1.tar.bz2 https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
!tar -xf $output_path/LJSpeech-1.1.tar.bz2 -C $output_path

--2025-07-17 18:24:30--  https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
Resolving data.keithito.com (data.keithito.com)... 195.181.163.196, 2400:52e0:1a02::1210:1
Connecting to data.keithito.com (data.keithito.com)|195.181.163.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2748572632 (2.6G) [text/plain]
Saving to: ‘/content/LJSpeech-1.1.tar.bz2’

/content/LJSpeech-1 100%[===================>]   2.56G   190MB/s    in 30s     

2025-07-17 18:25:00 (87.9 MB/s) - ‘/content/LJSpeech-1.1.tar.bz2’ saved [2748572632/2748572632]

^C


In [27]:
from TTS.config import BaseDatasetConfig

# Now define your dataset configuration
dataset_config = BaseDatasetConfig(
    formatter="ljspeech",
    meta_file_train="metadata.csv",
    path="/content/LJSpeech-1.1/" # Updated path to the extracted dataset
)

## ✅ Train a new model

Let's kick off a training run 🚀🚀🚀.

Deciding on the model architecture you'd want to use is based on your needs and available resources. Each model architecture has it's pros and cons that define the run-time efficiency and the voice quality.
We have many recipes under `TTS/recipes/` that provide a good starting point. For this tutorial, we will be using `GlowTTS`.

We will begin by initializing the model training configuration.

In [18]:
import os
from TTS.tts.configs.glow_tts_config import GlowTTSConfig

# --- Make sure these variables are defined first ---

# 1. Define the output path for your training run
output_path = "glow_tts_training_run"

# 2. Make sure `dataset_config` is defined from a previous cell
# Example:
# dataset_config = {
#     "name": "ljspeech",
#     "meta_file_train": "metadata.csv",
#     "path": "/content/MyTTSDataset/"
# }


# --- Now, create the model training configuration ---
config = GlowTTSConfig(
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=100,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    # Corrected path joining
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    # Use the variable defined above
    output_path=output_path,
    datasets=[dataset_config],
    save_step=1000,
)

Next we will initialize the audio processor which is used for feature extraction and audio I/O.

In [19]:
from TTS.utils.audio import AudioProcessor
ap = AudioProcessor.init_from_config(config)
# Modify sample rate if for a custom audio dataset:
# ap.sample_rate = 22050


 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


Next we will initialize the tokenizer which is used to convert text to sequences of token IDs.  If characters are not defined in the config, default characters are passed to the config.

In [64]:
from TTS.tts.utils.text.tokenizer import TTSTokenizer
tokenizer, config = TTSTokenizer.init_from_config(config)

Next we will load data samples. Each sample is a list of ```[text, audio_file_path, speaker_name]```. You can define your custom sample loader returning the list of samples.

In [63]:
from TTS.tts.datasets import load_tts_samples
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

 | > Found 13100 files in /content/LJSpeech-1.1


Now we're ready to initialize the model.

Models take a config object and a speaker manager as input. Config defines the details of the model like the number of layers, the size of the embedding, etc. Speaker manager is used by multi-speaker models.

In [65]:
from TTS.tts.models.glow_tts import GlowTTS
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

Trainer provides a generic API to train all the 🐸TTS models with all its perks like mixed-precision training, distributed training, etc.

In [28]:
from TTS.tts.datasets import load_tts_samples

# Load the training and evaluation samples from your dataset_config
# This returns two lists of file paths and text
train_samples, eval_samples = load_tts_samples(
    dataset_config,  # Use the corrected dataset_config
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

 | > Found 13100 files in /content/LJSpeech-1.1


### AND... 3,2,1... START TRAINING 🚀🚀🚀

In [57]:
from trainer import Trainer
import torch

# Create a dummy args object with necessary attributes
class Args:
    def __init__(self):
        self.restore_path = None
        self.use_cuda = torch.cuda.is_available()
        self.output_path = config.output_path
        self.config = config  # Add config to args
        self.config_path = None # Add config_path if needed by Trainer
        self.dataset_path = None # Add dataset_path if needed by Trainer
        self.continue_path = None # Add continue_path if needed by Trainer
        self.grad_accum_steps = 1 # Add grad_accum_steps attribute
        self.overfit_batch = False # Add overfit_batch attribute as seen in help(Trainer) example usage
        self.skip_train_epoch = False # Add skip_train_epoch attribute as seen in help(Trainer) example usage
        self.start_with_eval = False # Add start_with_eval attribute
        self.rank = 0 # Add rank attribute for single-process logging
        self.use_ddp = False # Add use_ddp attribute
        self.gpu = None # Add gpu attribute
        self.use_accelerate = False # Add use_accelerate attribute
        self.small_run = False # Add small_run attribute


args = Args()


# Initialize the trainer
trainer = Trainer(
    args, # First positional argument
    config, # Second positional argument
    config.output_path, # Third positional argument
    model=model, # Keyword argument
    parse_command_line_args=False, # Disable command-line argument parsing
    train_samples=train_samples, # Pass training samples
    eval_samples=eval_samples, # Pass evaluation samples
)

# Start training
trainer.fit()

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Num. of CPUs: 2
 | > Num. of Torch Threads: 1
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=glow_tts_training_run/run-July-17-2025_06+45PM-0000000
[!] Small Run, only using 0 samples.

 > Model has 28610257 parameters

 > EPOCH: 0/100
 --> glow_tts_training_run/run-July-17-2025_06+45PM-0000000
 ! Run is removed from glow_tts_training_run/run-July-17-2025_06+45PM-0000000
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/trainer/trainer.py", line 1833, in fit
    self._fit()
  File "/usr/local/lib/python3.11/dist-packages/trainer/trainer.py", line 1785, in _fit
    self.train_epoch()
  File "/usr/local/lib/python3.11/dist-packages/trainer/trainer.py", line 1483, in train_epoch
    self.train_loader = self.get_train_d



> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: gruut
| > Number of instances : 0
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/trainer/trainer.py", line 1833, in fit
    self._fit()
  File "/usr/local/lib/python3.11/dist-packages/trainer/trainer.py", line 1785, in _fit
    self.train_epoch()
  File "/usr/local/lib/python3.11/dist-packages/trainer/trainer.py", line 1483, in train_epoch
    self.train_loader = self.get_train_dataloader(
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/trainer/trainer.py", line 949, in get_train_dataloader
    return self._get_loader(
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/trainer/trainer.py", line 909, in _get_loader
    loader = model.get_data_loader(
             ^^^^^^^^^^^^^^^^^^^^^^
  File "

TypeError: object of type 'NoneType' has no len()

#### 🚀 Run the Tensorboard. 🚀
On the notebook and Tensorboard, you can monitor the progress of your model. Also Tensorboard provides certain figures and sample outputs.

In [59]:
!pip install tensorboard
!tensorboard --logdir=tts_train_dir

2025-07-17 18:48:40.239269: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752778120.276806   30517 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752778120.284818   30517 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-17 18:48:46.760245: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, u

## ✅ Test the model

We made it! 🙌

Let's kick off the testing run, which displays performance metrics.

We're committing the cardinal sin of ML 😈 (aka - testing on our training data) so you don't want to deploy this model into production. In this notebook we're focusing on the workflow itself, so it's forgivable 😇

You can see from the test output that our tiny model has overfit to the data, and basically memorized this one sentence.

When you start training your own models, make sure your testing data doesn't include your training data 😅

Let's get the latest saved checkpoint.

In [61]:
import glob, os
output_path = "tts_train_dir"
ckpts = sorted([f for f in glob.glob(output_path+"/*/*.pth")])
configs = sorted([f for f in glob.glob(output_path+"/*/*.json")])

In [ ]:
 !tts --text "Text for TTS" \
      --model_path $test_ckpt \
      --config_path $test_config \
      --out_path out.wav

## 📣 Listen to the synthesized wave 📣

In [ ]:
import IPython
IPython.display.Audio("out.wav")

## 🎉 Congratulations! 🎉 You now have trained your first TTS model!
Follow up with the next tutorials to learn more advanced material.

In [46]:
from trainer import Trainer

# Get the help information for the Trainer class
help(Trainer)

# Alternatively, you can try to inspect the constructor
# print(Trainer.__init__.__doc__)
# print(Trainer.__init__.__code__.co_varnames)

Help on class Trainer in module trainer.trainer:

class Trainer(builtins.object)
 |  Trainer(args: trainer.trainer.TrainerArgs, config: coqpit.coqpit.Coqpit, output_path: str, c_logger: trainer.logging.console_logger.ConsoleLogger = None, dashboard_logger: 'Logger' = None, model: torch.nn.modules.module.Module = None, get_model: Callable = None, get_data_samples: Callable = None, train_samples: List = None, eval_samples: List = None, test_samples: List = None, train_loader: torch.utils.data.dataloader.DataLoader = None, eval_loader: torch.utils.data.dataloader.DataLoader = None, training_assets: Dict = {}, parse_command_line_args: bool = True, callbacks: Dict[str, Callable] = {}, gpu: int = None) -> None
 |  
 |  Methods defined here:
 |  
 |  __init__(self, args: trainer.trainer.TrainerArgs, config: coqpit.coqpit.Coqpit, output_path: str, c_logger: trainer.logging.console_logger.ConsoleLogger = None, dashboard_logger: 'Logger' = None, model: torch.nn.modules.module.Module = None, get_